<a href="https://colab.research.google.com/github/MoroF10/enem2023-data-analysis/blob/main/03_An%C3%A1lise_Prescritiva_ENEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Análise Prescritiva dos dados do ENEM**

## Objetivo do Projeto<br>
Este notebook tem como foco a aplicação de modelos estatísticos para prever as notas de Matemática dos participantes do ENEM. A proposta é concentrar esforços exclusivamente na etapa de modelagem preditiva, utilizando dados que já foram tratados, limpos e explorados em notebooks anteriores disponíveis neste repositório.

Diferente de abordagens exploratórias, este projeto parte do pressuposto de que o entendimento dos dados — como distribuição, correlações e inconsistências — já foi realizado. O objetivo aqui é testar diferentes algoritmos de regressão, avaliar seus desempenhos e identificar o modelo mais eficiente para estimar as notas com base nas variáveis disponíveis.

A intenção é oferecer uma abordagem prática e objetiva, que possa ser útil em estudos educacionais, simulações ou como ferramenta de apoio à análise de desempenho.

## Estratégias Utilizadas na Análise <br>
Diante das características da base de dados — com muitas colunas, variáveis qualitativas e ausência de distribuição normal — algumas técnicas foram adotadas para otimizar o desempenho dos modelos:<br>
<br>
### Normalização<br>
Como as variáveis possuem escalas distintas (ex.: número de acertos, tempo de prova, renda familiar), será aplicada uma técnica de normalização. Isso garante que todas as variáveis estejam na mesma faixa de valores, evitando que atributos com maior magnitude dominem o processo de aprendizagem e favorecendo algoritmos sensíveis à escala.<br>
<br>
### PCA (Análise de Componentes Principais)<br>
Para lidar com a alta dimensionalidade, será utilizada a técnica de PCA com os seguintes objetivos:

. Eliminar redundâncias e correlações entre variáveis

. Reduzir o tempo de treinamento dos modelos

. Minimizar o risco de overfitting

. Preservar a maior parte da variância explicada pelos dados originais

Essa redução permite trabalhar com um conjunto de variáveis mais compacto e informativo, facilitando a construção de modelos mais eficientes e interpretáveis.<br>
<br>
### Transformação de Variáveis Qualitativas<br>

A base contém diversas colunas com atributos categóricos. Para convertê-las em variáveis numéricas, será utilizada a técnica de SimpleImputer do scikit-learn, evitando o uso de OneHotEncoder, que poderia gerar um conjunto de dados excessivamente grande e esparso. Essa escolha busca manter a eficiência computacional sem perder representatividade.

## Importando bibliotecas e a base de dados

In [83]:
#Importando as bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format','{:.2f}'.format)

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae

import warnings
warnings.filterwarnings('ignore')

In [39]:
#Importando os dados
caminho = ("dados_amostra.csv")
df = pd.read_csv(caminho, sep=',', encoding='latin1')

In [3]:
#Visualizando os dados
df.head()

,NU_INSCRICAO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,CO_UF_ESC,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,CO_MUNICIPIO_PROVA,NO_MUNICIPIO_PROVA,CO_UF_PROVA,SG_UF_PROVA,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,TP_PRESENCA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,TP_LINGUA,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,n_faltas
0,210061362356,8,F,1,3,1,1,5,1,1.00,0,0.00,0,0.00,0,2.00,1.00,0.00,3143302,Montes Claros,31,MG,1,1,1,1,481.30,295.50,503.80,383.00,1,1.00,120.00,120.00,120.00,120.00,80.00,560.00,H,H,A,A,3,B,A,B,D,B,A,B,A,A,A,A,A,A,A,A,B,D,A,A,B,4
1,210060716718,5,F,1,2,1,1,2,1,1.00,0,0.00,0,0.00,0,2.00,1.00,0.00,3304557,Rio de Janeiro,33,RJ,1,1,1,1,590.60,604.80,543.30,650.40,0,1.00,120.00,40.00,40.00,80.00,40.00,320.00,E,D,C,B,3,C,A,C,C,A,A,B,B,B,A,B,A,A,B,A,A,D,A,B,B,4
2,210058482642,1,F,1,3,1,3,0,1,1.00,1,0.00,0,0.00,0,2.00,1.00,0.00,2918209,JiquiriÃÂ§ÃÂ¡,29,BA,1,1,1,1,421.50,489.40,444.10,397.00,0,1.00,80.00,100.00,80.00,100.00,80.00,440.00,B,E,A,A,3,B,A,B,B,A,A,B,A,A,A,A,A,A,A,A,A,B,A,A,A,4
3,210059249770,3,F,1,1,1,1,1,1,1.00,0,0.00,0,0.00,0,2.00,1.00,0.00,3534401,Osasco,35,SP,1,1,1,1,594.30,618.20,662.10,670.10,0,1.00,160.00,200.00,180.00,180.00,180.00,900.00,D,G,D,E,2,O,A,C,D,B,A,B,B,B,B,B,B,B,C,A,B,C,A,C,B,4
4,210059793602,2,F,0,0,1,2,0,2,1.00,0,2307650.00,MaracanaÃÂº,23.00,CE,2.00,1.00,1.00,2307650,MaracanaÃÂº,23,CE,1,1,1,1,0.00,360.00,565.10,0.00,1,1.00,120.00,120.00,120.00,140.00,80.00,580.00,B,E,C,B,5,D,A,C,D,B,A,B,A,B,A,B,A,A,B,B,A,D,A,A,B,4


## Transformando os dados qualitativos

A etapa inicial do processamento dos dados será a conversão de variáveis qualitativas em valores numéricos. Essa transformação é essencial para garantir o funcionamento adequado dos modelos de machine learning. Para isso, utilizarei a biblioteca scikit-learn, especificamente o OrdinalEncoder.

In [42]:
#Criando uma lista com todos os valores categoricos
cat_cols = df.select_dtypes(include=['object', 'category']).columns

#criando uma instância para Label Enconder
le = LabelEncoder()

#Aplicando Label Enconder em todos os dados categoricos
for col in cat_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

In [44]:
#Visualizando os dados
df.head()

,NU_INSCRICAO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,CO_UF_ESC,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,CO_MUNICIPIO_PROVA,NO_MUNICIPIO_PROVA,CO_UF_PROVA,SG_UF_PROVA,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,TP_PRESENCA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,TP_LINGUA,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,n_faltas
0,210061362356,8,0,1,3,1,1,5,1,1.00,0,0.00,0,0.00,0,2.00,1.00,0.00,3143302,436,31,10,1,1,1,1,481.30,295.50,503.80,383.00,1,1.00,120.00,120.00,120.00,120.00,80.00,560.00,7,7,0,0,3,1,0,1,3,1,0,1,0,0,0,0,0,0,0,0,1,3,0,0,1,4
1,210060716718,5,0,1,2,1,1,2,1,1.00,0,0.00,0,0.00,0,2.00,1.00,0.00,3304557,579,33,18,1,1,1,1,590.60,604.80,543.30,650.40,0,1.00,120.00,40.00,40.00,80.00,40.00,320.00,4,3,2,1,3,2,0,2,2,0,0,1,1,1,0,1,0,0,1,0,0,3,0,1,1,4
2,210058482642,1,0,1,3,1,3,0,1,1.00,1,0.00,0,0.00,0,2.00,1.00,0.00,2918209,358,29,4,1,1,1,1,421.50,489.40,444.10,397.00,0,1.00,80.00,100.00,80.00,100.00,80.00,440.00,1,4,0,0,3,1,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,4
3,210059249770,3,0,1,1,1,1,1,1,1.00,0,0.00,0,0.00,0,2.00,1.00,0.00,3534401,468,35,25,1,1,1,1,594.30,618.20,662.10,670.10,0,1.00,160.00,200.00,180.00,180.00,180.00,900.00,3,6,3,4,2,14,0,2,3,1,0,1,1,1,1,1,1,1,2,0,1,2,0,2,1,4
4,210059793602,2,0,0,0,1,2,0,2,1.00,0,2307650.00,177,23.00,6,2.00,1.00,1.00,2307650,411,23,5,1,1,1,1,0.00,360.00,565.10,0.00,1,1.00,120.00,120.00,120.00,140.00,80.00,580.00,1,4,2,1,5,3,0,2,3,1,0,1,0,1,0,1,0,0,1,1,0,3,0,0,1,4


## Normalizando os dados

Neste momento, aplicarei a técnica de normalização dos dados por meio do StandardScaler, que ajusta as variáveis para uma mesma escala. Essa padronização é fundamental para garantir que atributos com diferentes unidades de medida tenham impacto equilibrado no desempenho dos modelos de machine learning.

A partir desta etapa, deixarei de utilizar o conjunto de dados original, pois pretendo utilizar as notas da prova de matemática como variável alvo da predição. Como esse valor não deve ser alterado de forma alguma, é essencial mantê-lo em seu estado original. Por isso, irei separá-lo em uma nova variável chamada Y.

In [47]:
#criando uma cópia do conjunto de dados
X = df.copy()

#Selecionando as notas de matemática
y = df['NU_NOTA_MT']

#removendo a variável alvo do conjunto de dados
X.drop('NU_NOTA_MT', axis=1, inplace=True)

In [55]:
#Instanciar o scaler
scaler = StandardScaler()

# Aplicar o fit_transform para normalizar
X_scaled = scaler.fit_transform(X)

#Transformar de volta para DataFrame com os mesmos nomes de colunas
X = pd.DataFrame(X_scaled, columns=X.columns)

In [57]:
#Visualizando os dados
X.head()

,NU_INSCRICAO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,CO_UF_ESC,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,CO_MUNICIPIO_PROVA,NO_MUNICIPIO_PROVA,CO_UF_PROVA,SG_UF_PROVA,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,TP_PRESENCA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,TP_LINGUA,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,n_faltas
0,1.13,1.08,-0.79,-0.08,0.97,-0.14,-1.06,0.85,-0.75,-0.04,-0.50,-0.58,-0.52,-0.58,-0.53,-0.26,-0.11,-0.60,0.03,0.20,0.01,-0.53,0.22,0.02,0.02,0.22,0.08,-2.35,-0.15,1.13,-0.20,-0.01,-0.36,0.03,-0.21,-0.46,-0.25,1.78,1.92,-1.43,-1.32,-0.56,-0.76,-0.29,-0.64,0.87,0.51,-0.55,-0.14,-0.90,-1.33,-0.41,-0.98,-0.20,-0.59,-1.57,-0.41,1.78,0.39,-0.34,-0.81,0.30,0.23
1,0.58,0.21,-0.79,-0.08,-0.01,-0.14,-1.06,0.06,-0.75,-0.04,-0.50,-0.58,-0.52,-0.58,-0.53,-0.26,-0.11,-0.60,0.19,0.84,0.20,0.53,0.22,0.02,0.02,0.22,0.86,0.87,0.33,-0.88,-0.20,-0.01,-1.90,-1.78,-1.09,-1.11,-1.34,0.20,-0.51,-0.17,-0.65,-0.56,-0.50,-0.29,0.58,-0.38,-0.86,-0.55,-0.14,0.78,0.69,-0.41,0.94,-0.20,-0.59,-0.43,-0.41,-0.56,0.39,-0.34,0.26,0.30,0.23
2,-1.31,-0.95,-0.79,-0.08,0.97,-0.14,1.54,-0.46,-0.75,-0.04,2.01,-0.58,-0.52,-0.58,-0.53,-0.26,-0.11,-0.60,-0.19,-0.15,-0.19,-1.33,0.22,0.02,0.02,0.22,-0.34,-0.33,-0.87,-0.88,-0.20,-1.07,-0.75,-0.87,-0.65,-0.46,-0.79,-1.39,0.10,-1.43,-1.32,-0.56,-0.76,-0.29,-0.64,-1.63,-0.86,-0.55,-0.14,-0.90,-1.33,-0.41,-0.98,-0.20,-0.59,-1.57,-0.41,-0.56,-1.49,-0.34,-0.81,-3.29,0.23
3,-0.66,-0.37,-0.79,-0.08,-0.99,-0.14,-1.06,-0.20,-0.75,-0.04,-0.50,-0.58,-0.52,-0.58,-0.53,-0.26,-0.11,-0.60,0.42,0.34,0.40,1.46,0.22,0.02,0.02,0.22,0.89,1.01,1.77,-0.88,-0.20,1.05,1.17,1.39,1.12,1.16,1.28,-0.33,1.31,0.47,1.34,-1.33,2.64,-0.29,0.58,0.87,0.51,-0.55,-0.14,0.78,0.69,2.41,0.94,4.90,1.71,0.70,-0.41,1.78,-0.55,-0.34,1.34,0.30,0.23
4,-0.20,-0.66,-0.79,-3.01,-1.97,-0.14,0.24,-0.46,0.82,-0.04,-0.50,0.93,1.33,0.93,0.21,-0.26,-0.11,1.57,-0.79,0.09,-0.79,-1.20,0.22,0.02,0.02,0.22,-3.35,-1.68,0.60,1.13,-0.20,-0.01,-0.36,0.03,0.23,-0.46,-0.16,-1.39,0.10,-0.17,-0.65,0.99,-0.24,-0.29,0.58,0.87,0.51,-0.55,-0.14,-0.90,0.69,-0.41,0.94,-0.20,-0.59,-0.43,2.42,-0.56,0.39,-0.34,-0.81,0.30,0.23


## Aplicando o PCA

Devido ao tamanho elevado do meu conjunto de dados, optei por aplicar a técnica de PCA (Principal Component Analysis) como uma etapa de redução de dimensionalidade. O PCA é fundamental nesse contexto, pois transforma as variáveis originais em um novo conjunto de componentes principais que retêm a maior parte da variância dos dados, com menos dimensões.

Essa abordagem oferece diversas vantagens: além de reduzir o tempo de processamento e o uso de memória, ela também pode melhorar o desempenho dos modelos de machine learning, minimizando o risco de overfitting e eliminando redundâncias entre variáveis altamente correlacionadas. Com isso, o modelo se torna mais eficiente e interpretável, sem perder informações relevantes para a tarefa de predição.

In [65]:
#Instanciar PCA
pca = PCA(n_components=0.95)  # Retém 95% da variância

#Aplicar PCA
X_pca = pca.fit_transform(X)

# Converter para DataFrame
X_df = pd.DataFrame(X_pca, columns=component_names)

Após a aplicação do PCA, o conjunto de dados foi reduzido de 63 para 42 colunas, representando uma diminuição significativa na dimensionalidade. Essa redução contribui diretamente para a melhoria do tempo de processamento. A redução dimensional via PCA permite que os algoritmos operem com maior desempenho, sem comprometer a representatividade das informações mais relevantes.

In [67]:
#visualização dos dados
X_df.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20,PC21,PC22,PC23,PC24,PC25,PC26,PC27,PC28,PC29,PC30,PC31,PC32,PC33,PC34,PC35,PC36,PC37,PC38,PC39,PC40,PC41,PC42
0,-2.39,1.01,-0.55,0.60,0.13,0.13,-0.38,-0.13,0.40,-1.34,1.81,-0.27,0.27,0.13,0.61,1.23,-0.17,-0.71,-0.41,-0.72,-0.32,-1.16,1.25,-0.57,-0.74,1.11,-1.15,0.40,1.23,1.71,0.07,-2.29,-0.05,-0.88,0.50,0.49,0.20,-0.62,-1.20,-1.16,-0.08,-0.67
1,-1.48,-0.29,-1.49,1.76,1.20,-0.42,0.99,0.90,1.02,1.03,0.40,-0.42,0.05,-0.22,0.25,0.27,0.37,0.60,-1.40,0.12,0.14,-0.35,-0.99,0.39,0.12,-0.26,-0.02,-0.27,0.72,-0.14,0.49,0.17,0.81,0.61,-1.30,-0.06,-0.28,-0.54,0.07,-0.68,0.65,0.37
2,-4.42,0.54,0.20,0.34,-2.63,-0.65,1.16,0.09,-2.92,0.54,-1.75,0.67,-0.27,0.04,-0.88,-0.16,0.02,0.10,0.29,0.79,0.15,0.87,-0.03,-0.21,-0.19,-1.23,-0.36,-0.41,1.72,0.63,-0.16,-0.47,0.46,-0.19,-0.24,0.25,0.06,0.09,0.15,-0.92,0.25,0.08
3,5.21,2.39,-1.92,-0.21,1.63,-0.18,-0.23,1.48,-1.63,0.28,-0.20,1.04,2.01,-0.89,-0.08,-1.99,1.15,1.11,-0.64,-1.40,-1.96,-0.35,-0.99,-1.17,-0.18,0.61,-1.13,-1.58,-0.80,0.52,-1.90,-0.30,0.23,-0.50,-0.06,0.24,-0.67,-0.16,-0.42,0.08,0.68,0.03
4,-0.25,-2.19,1.70,-0.81,-1.13,0.59,-0.41,-1.32,-0.36,-0.20,0.84,-1.75,0.49,0.21,0.81,-1.05,1.25,0.83,0.33,1.74,0.68,-1.95,0.08,-0.04,-0.74,-0.12,0.87,2.60,-0.23,-1.10,-1.75,0.04,0.58,0.47,0.39,-0.80,0.58,-0.72,-0.78,-0.83,-0.24,0.15


## Modelagem Preditiva das Notas de Matemática

Com os dados devidamente tratados e a dimensionalidade reduzida por meio do PCA, iniciarei agora o processo de predição utilizando técnicas de aprendizado de máquina. Nesta etapa, o objetivo é avaliar o desempenho de diferentes algoritmos na tarefa de prever as notas da prova de matemática.

Para isso, serão testados alguns modelos amplamente utilizados e eficazes em problemas de regressão, como:

**Árvore de Decisão**: modelo interpretável que segmenta os dados com base em regras simples, ideal para entender os critérios de decisão.

**Random Forest**: conjunto de múltiplas árvores de decisão que melhora a precisão e reduz o risco de overfitting.

**Regressão Linear**: abordagem clássica que busca uma relação linear entre as variáveis preditoras e o alvo.

A comparação entre esses modelos permitirá identificar qual deles oferece melhor desempenho para este conjunto de dados, considerando métricas como MAE (Mean Absolute Error) e tempo de execução.

Como etapa inicial do processo de modelagem, é fundamental realizar a separação dos dados em conjuntos de treino e teste. Essa divisão permite avaliar o desempenho dos modelos de forma mais confiável, evitando que eles sejam testados nos mesmos dados usados para o aprendizado. Para isso, utilizarei a função train_test_split da biblioteca scikit-learn, que facilita essa tarefa de maneira prática e eficiente.

In [74]:
#Separando os dados entre treino e test
X_train, X_valid, y_train, y_valid = train_test_split(X_df, y, random_state=42)

Como as notas de matemática representam valores numéricos contínuos, o problema em questão é do tipo regressão. Por esse motivo, utilizarei bibliotecas específicas para esse tipo de tarefa, como as mencionadas anteriormente: Regressão Linear, Árvore de Decisão e Random Forest, todas voltadas para modelos de regressão.

In [88]:
#importando as bibliotecas
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression

In [94]:
#Instanciando o modelo Regressão Linear
lr = LinearRegression(
    fit_intercept=True,
    copy_X=True,
    n_jobs=-1,
    positive=False
)

#Treinando o modelo
lr.fit(X_train, y_train)

#Criando uma previsão dos dados
y_pred = lr.predict(X_valid)

#Medindo a taxa de erro
mae = mean_absolute_error(y_valid, y_pred)
print(f"MAE: {mae}")

MAE: 68.82547985975685


In [91]:
#Instanciando o modelo Árvore de Decisão
dt = DecisionTreeRegressor(
    criterion='squared_error',
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42
)

#Treinando o modelo
dt.fit(X_train, y_train)

#Criando uma previsão dos dados
y_pred = dt.predict(X_valid)

#Medindo a taxa de erro
mae = mean_absolute_error(y_valid, y_pred)
print(f"MAE: {mae}")

MAE: 96.34730837388182


In [93]:
#Instanciando o modelo Random Forest
rf = RandomForestRegressor(
    n_estimators=200,
    max_depth=15,
    min_samples_split=4,
    min_samples_leaf=2,
    max_features='sqrt',
    bootstrap=True,
    random_state=42,
    n_jobs=-1
)

#Treinando o modelo
rf.fit(X_train, y_train)

#Criando uma previsão dos dados
y_pred = rf.predict(X_valid)

#Medindo a taxa de erro
mae = mean_absolute_error(y_valid, y_pred)
print(f"MAE: {mae}")

MAE: 84.60030912469334


A partir da análise dos valores de MAE obtidos, é possível identificar que o modelo de Regressão Linear apresentou o melhor desempenho entre os modelos testados. Como o MAE representa o erro médio absoluto entre os valores reais e previstos, quanto mais próximo de zero estiver esse valor, melhor será a qualidade das previsões realizadas pelo modelo.